In [99]:
import numpy as np
import pandas as pd
import csv
import warnings

In [100]:
warnings.filterwarnings("ignore")

In [10]:
trainV = pd.read_csv("training_variants.csv")

In [11]:
trainV.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [12]:
trainV.shape

(3321, 4)

In [103]:
def read_this_funky_csv(source):
  # be sure to pass a source object that supports
  # iteration (e.g. a file object, or a list of csv text lines)
  return csv.reader((line.replace('||', '|') for line in source), delimiter='|')

In [104]:
#trainT = pd.read_csv('training_text',sep="||")

fileHandle = open('training_text', 'r')

In [120]:
pipereader = read_this_funky_csv(fileHandle)

In [133]:
fileHandle.seek(0)

In [134]:
columns = next(pipereader)

In [127]:
newcolumns = columns[0].split(',')

In [128]:
df = pd.DataFrame(columns=newcolumns)

In [141]:
ls = next(pipereader)

'5'

In [173]:
dict(zip(ls[0],ls[1]))

{'5': 'O'}

In [164]:
nls = list(t)

In [155]:
test = pd.DataFrame(data=nls)
test

,0
0,5
1,Oncogenic mutations in the monomeric Casitas B...


In [136]:
#df2 = pd.DataFrame(next(pipereader),columns=newcolumns)
#df2
#for row in pipereader:

In [102]:
trainT.head(10)

,"ID,Text"
0,0||Cyclin-dependent kinases (CDKs) regulate a ...
1,1|| Abstract Background Non-small cell lung c...
2,2|| Abstract Background Non-small cell lung c...
3,3||Recent evidence has demonstrated that acqui...
4,4||Oncogenic mutations in the monomeric Casita...
5,5||Oncogenic mutations in the monomeric Casita...
6,6||Oncogenic mutations in the monomeric Casita...
7,7||CBL is a negative regulator of activated re...
8,8|| Abstract Juvenile myelomonocytic leukemia ...
9,9|| Abstract Juvenile myelomonocytic leukemia ...
